In [71]:
import numpy as np
import math

In [274]:
"""
Esta função é utilizada para reduzir a propogação de erros de 
arredondamento oriundos da aritmética com pontos flutuantes
"""
def truncador(numero):
    if(type(numero)) == float:
        numero_str = str(numero)
        inteira, decimal = numero_str.split('.')
        digitos_decimal = list(decimal)    
        if len(digitos_decimal) >= 3:
            if digitos_decimal[0] == '9' and digitos_decimal[1] == '9' and digitos_decimal[2] == '9':
                return math.ceil(numero)
        return math.floor(numero*10**3)/(10**3)
    else:
        return numero

In [4]:
"""
Esta função nos retorna a derivada particial de uma 
função multivariada em relação à uma variável de índice i
"""
def derivada(f, X, i):
    h = 10**-10
    Xh = []
    for j in range(len(X)):
        if j != i:  
            Xh.append(X[j])
        else:
            Xh.append(X[i] + h)
    return (f(Xh) - f(X))/h

In [106]:
"""
Esta função nos retorna o gradiente completo de uma função
aplicada sobre um determinado ponto
"""
def gradiente(f, X):
    grad = []
    for i in range(len(X)):
        grad.append(truncador(derivada(f, X, i)))
    return np.array(grad)

In [275]:
"""
Esta função (extra) pode ser usada para verificar se uma matriz é positiva definida,
ou seja, pode ser utilizada em verificações de otimalidade (esta função não foi utilizada neste estudo)
"""
def cholesky_positiva_semidef(A, n):
    l = np.zeros((n, n))
    y = np.zeros((n))
    X = np.zeros((n))
    
    if (A[0][0] >= 0):
        l[0][0] = math.sqrt(A[0][0])
    else:
        return False
    
    for j in range(1, n):        
        l[j][0] = A[j][0]/l[0][0]
    
    for i in range(1, n-1):
        sum_col = 0
        for k in range(0, i):
            sum_col = sum_col + math.pow(l[i][k], 2)
        if A[i][i] >= 0:
            l[i][i] = math.sqrt(A[i][i] - sum_col)
        else:
            return False
        
        for j in range(i+1, n):
            sum_prod = 0
            for k in range(0, i): sum_prod = sum_prod + l[j][k]*l[i][k]
            l[j][i] = (A[j][i] - sum_prod)/l[i][i]
            
    sum_col_last = 0
    for k in range(0, n-1): sum_col_last = sum_col_last + math.pow(l[n-1][k], 2) 
    
    if(A[n-1][n-1] >=0 ):
        l[n-1][n-1] = math.sqrt(A[n-1][n-1] - sum_col_last)
    else:
        return False
    
    return True

In [130]:
"""
Esta função calcula a solução de um sistema linear Mx = b, M quadrada, a partir de sua matriz aumentada A = [M | b] 
"""
def pivotamento_parcial(A, n):
    X = [0]*n
    nrow = [i for i in range(0, n)]
    
    for i in range(0, n):
        p = i
        pivo_maior =  math.fabs(A[nrow[p]][i])
        for j in range(i, n):
            candidato = math.fabs(A[nrow[j]][i])
            if( candidato > pivo_maior ): 
                pivo_maior = candidato
                p = j
        
        if(A[nrow[p]][i] == 0): return 'Sistema sem solução única'
            
        if(nrow[i] != nrow[p]):
            ncopy = nrow[i]
            nrow[i] = nrow[p]
            nrow[p] = ncopy
        
        for j in range(i+1, n):
            m = A[nrow[j]][i]/A[nrow[i]][i]
            mA_nrowi = [m*a for a in A[nrow[i]]]
            
            A[nrow[j]] = [a_nrowj - ma_nrowi for a_nrowj, ma_nrowi in zip(A[nrow[j]], mA_nrowi)]
            
    if(A[nrow[n-1]][n-1] == 0): return 'Sistema sem solução única'
    
    X[n-1] = A[nrow[n-1]][n]/A[nrow[n-1]][n-1]
    
    for i in range(n-2, -1, -1):
        sum_lin = 0
        for j in range(i, n):
            sum_lin = sum_lin + A[nrow[i]][j]*X[j]
        X[i] = ( A[nrow[i]][n] - sum_lin ) / A[nrow[i]][i]

    return X

A função seguinte retorna uma aproximação para a derivada segunda de uma função em um ponto a partir da seguinte fórmula:

$$
\dfrac{f(x_0 + \hat{h}, y_0 + \hat{h}) - f(x_0, y_0 + \hat{h}) - f(x_0  + \hat{h}, y_0) + f(x_0, y_0)}{\hat{h}^2}
$$

Que pode ser obtida através da expansão de:

$$
\dfrac{\partial^2 f(x_0, y_0)}{\partial x \partial y} \approx \dfrac{f_{x}(x_0, y_0 + \hat{h}) - f_{x}(x_0, y_0)}{\hat{h}}
$$

Onde $f_{x}(x_0, y_0)$ representa a derivada parcial de $f$ em relação à $x$ no ponto $(x_0, y_0)$, e onde tomamos sempre o mesmo número pequeno $\hat{h}$ - para facilitar os cálculos.
E extensão para funções de mais de duas variáveis é trivial.

In [62]:
"""
Retorna a linha i da matriz hessiana H
"""
def derivada_segunda(f, X, i, j):
    h = 10**-6
    Xhh = []
    Xh_ = []
    Xh = []
    for k in range(len(X)):
        if k == i and k == j:
            Xh.append(X[k] + h)
            Xh_.append(X[k] + h)
        elif k == i and k != j:
            Xh.append(X[k] + h)
            Xh_.append(X[k])
        elif k != i and k == j:
            Xh.append(X[k])
            Xh_.append(X[k] + h)
        else:
            Xh.append(X[k])
            Xh_.append(X[k])
            
    for k in range(len(Xh)):
        if k == j:
            Xhh.append(Xh[k] + h)
        else:
            Xhh.append(Xh[k])
            
    return (f(Xhh) - f(Xh_) -f(Xh) + f(X))/(h**2)

In [63]:
"""
Retorna a matriz hessiana H
"""
def hessiana(f, X):
    n = len(X)
    H = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            H[i][j] = truncador(derivada_segunda(f, X, i, j))
    return H

In [185]:
"""
Esta função deve receber a transposta de Z, pois ortogonaliza vetores linha.
Args: Z.T, onde o tipo de Z é numpy.ndarray
Exemplo:
    Z = np.array([[1, 3],[2, 4]])
    a, b = gram_schmidt(Z.T)
"""
def gram_schmidt(Z_no):
    Z_no = np.array(Z_no)
    U = []
    U.append(list(Z_no[0]))
    for i in range(1, len(Z_no)):
        u = list(Z_no[i])
        soma_projecoes = 0
        for j in range(i):
            soma_projecoes += (np.dot(Z_no[i], U[j])/np.dot(U[j], U[j]))*np.array(U[j])
        u = list(u - soma_projecoes)
        U.append(u)
    return U

## Algoritmos

O método do gradiente apresentado por Ana Friedlander é simples, e pode ser implementado com uma recursão e uma regra de parada definida pelo usuário da função

In [255]:
def metodo_gradiente(f, Xk, K, K_max, TOL):
    if K > K_max:
        return False
    grad_f = gradiente(f, Xk)
    dk = -grad_f
    alpha = 0.01
    t = 0.5
    while( f(Xk + t*dk) > f(Xk) + t*alpha*np.dot(grad_f, dk)):
        t = 0.5*t
    Xk1 = Xk + t*dk
    for i in range(len(Xk)):
        if math.fabs(Xk1[i] - Xk[i]) > TOL:
            return metodo_gradiente(f, Xk1, K + 1, K_max, TOL)
    return Xk

No método de newton, devemos resolver o sistema linear:

$$
\nabla^2 f(X_k) d_k = - \nabla f(X_k)
$$

Para encontrarmos a direção de Newton $d_k$. Este é um bom caso de uso para a função de decomposição de cholesky (o método de Newton deve ser utilizado caso a Hessiana de $f$ seja positiva definida). A solução do sistema linear acima é realizada pelo método de eliminação de gauss com pivotamento parcial, que reduz problemas de erros de arredondamento oriundos da aritmética com pontos flutuantes. A resolução de um sistema via eliminação de gauss tem complexidade $O(n^3)$, e o total de computações adicionais pelo pivotamento parcial é pequeno - como pode ser encontrado em Burden, Numerical Analaysis.

In [145]:
def metodo_newton(f, Xk, K, K_max, TOL):
    if K > K_max:
        return False
    H = hessiana(f, Xk)
    grad_f = gradiente(f, Xk)
    H_aumentada = []
    for i in range(len(H)):
        H_aumentada.append([])
        for j in range(len(H[i])):
            H_aumentada[i].append(H[i][j])
        H_aumentada[i].append(-grad_f[i])

    dk = np.array(pivotamento_parcial(H_aumentada, len(H_aumentada)))
    alpha = 0.01
    t = 0.5
    while( f(Xk + t*dk) > f(Xk) + t*alpha*np.dot(grad_f, dk)):
        t = 0.5*t
    Xk1 = Xk + t*dk
    for i in range(len(Xk)):
        if math.fabs(Xk1[i] - Xk[i]) > TOL:
            return metodo_newton(f, Xk1, K + 1, K_max, TOL)
    return Xk

Consideramos um problema não linear da forma:

$$
\min f(x)\\
\text{sa:} \ Ax = b
$$

Onde $A \in R^{m \ \text{x} \ n}$, $ 1 \leq m < n$, posto de $A = m$, $f : R^{n} \to R$, $x \in R^{n}$ e $b \in R^{n}$.

Na página 57, Ana F. sugere um algoritmo seguindo a direção do gradiente projetado no espaço factível, a partir de uma matriz de projetção $Proj_{Nu(A)}$ que projeta um vetor $v$ no núcleo de $A$. O raciocínio para encontrar a primeira fórmula pode ser encontrado em Luenberguer. De forma simplificada, a ideia é que de $Ad = 0$ - pois a direção factível deve estar no núcleo de $A$ - segue que $d \perp Im(A^t)$. Daí, qualquer vetor de $R^n$ pode ser escrito como uma combinação do vetor $d$ e de um vetor da forma $A^{t}\lambda$, onde $\lambda \in R^m$. Daí, segue que $-\nabla f(x) = d + A^{t}\lambda$, para algum $\lambda \in R^m$, pois $\nabla f(x) \in R^n$. Multiplicando à esquerda ambos os lados da equação por $A$, temos que $-A\nabla f(x) = Ad + (AA^{t})\lambda$. Como $d$ está no núcleo de $A$, segue que $Ad = 0$, e, portanto, isolando o $\lambda$, obtemos $\lambda = - (AA^{t})^{-1}A\nabla f(x)$. 

O problema de utilizarmos diretamente a matriz $A$, é que a necessidade de calcular a inversa de $AA^{t}$ adiciona um custo computacional excessivo em problemas grandes. Por isso, podemos usar a outra fórmula sugerida por Ana F., ortogonalizando a matriz $Z$ cujas colunas geram $Nu(A)$ através do processo de gram-schmidt. A lógica para encontrar a fórmula sugerida por Ana F. é semelhante à do parágrafo anterior: segue de $Ad = 0 \Rightarrow \exists \gamma \in R^{n \text{x} m} \ \text{tal que} \ d = Z\gamma$ que:
$$
-\nabla f(x) = d + A^{t}\lambda \Rightarrow -Z^{t}\nabla f(x) = Z^{t}Z\gamma + Z^{t}A^{t}\lambda = Z^{t}Z\gamma
$$ 

pois $Z\gamma = d$ e $Z^{t}A^{t}\lambda = 0$, pois o vetor $A^{t}\lambda$ é ortogonal ao $Nu(A)$. Daí, basta isolarmos o $\gamma$ para encontrarmos $\gamma = -(Z^{t}Z)^{-1}Z^{t}\nabla f(x)$, e, multiplicando à esquerda por $Z$:

$$
d = -Z(Z^{t}Z)^{-1}Z^{t}\nabla f(x)
$$

Que se torna:

$$
d = -ZZ^{t}\nabla f(x)
$$

Quando Z é ortogonal. Notemos que a regra de parada do algoritmo funciona porque no ponto ótimo, o gradiente da função $f$ será ortogonal ao núcleo de $A$ - o que implica que o gradiente é uma combinação linear das linhas de $A$ - e, portando, sua projeção no núcleo terá norma zero.

In [276]:
"""
Esta função recebe a matriz Z, cujas colunas geram o núcleo da matriz de restrições A
"""
def metodo_gradiente_projetado(f, Xk, Z, Proj, t, K, K_max, TOL):
    if K > K_max:
        return False
    if K == 0:
        if len(Z.T > 1):
            Z = np.array(gram_schmidt(Z.T)).T
        Proj = np.matmul(Z, Z.T)
    grad_f = gradiente(f, Xk)
    grad_proj = np.matmul(Proj, grad_f)
    if np.linalg.norm(grad_proj) > TOL:
        t = t*0.5
        return metodo_gradiente_projetado(f, Xk - t*grad_proj, Z, Proj, t, K + 1, K_max, TOL)
    return Xk

## Funções para um breve Estudo

In [9]:
def f2(X):
    return X[0]**2 + X[1]**2

O mínimo para essa simples função é em $(0, 0)$

In [260]:
metodo_gradiente(f2, [80, 2], 0, 100, 0.001)

array([-5.0209531e-11, -4.9008397e-11])

In [10]:
def f1(X):
    return X[0]**3 + X[1]**4

Notemos que essa função não tem mínimo, pois ela decresce ao infinito quando seu primeiro termo decresce ao infinito. O resultado é que o algoritmo retorna o resultado da iteração máxima, ou um resultado de módulo grande quando o algoritmo dá passos muito pequenos.

In [265]:
metodo_gradiente(f1, [1, 1], 0, 100, 0.001)

array([-4.59002033e+08,  1.64409680e-01])

In [271]:
def fxy(X):
    return X[0]**4 + X[1]**2 + 1

Para esta função, os algoritmos do gradiente e de newton foram testados, alcançando o mesmo resultado rapidamente

In [272]:
metodo_gradiente(fxy, [1, 2], 0, 50, 0.01)

array([8.2740371e-08, 0.0000000e+00])

In [273]:
metodo_newton(fxy, [1, 2], 0, 50, 0.01)

array([5.40489905e-02, 2.92831934e-05])

In [220]:
def fx(X):
    return X[0]**2 + X[1]**2
Z_fx = np.array([[1], [-1]])

O método do gradiente projetado funciona bem para este sistema simples, cuja resposta real é mínimo em $(0, 0)$

In [223]:
metodo_gradiente_projetado(fx, [1 , 0], Z_fx, [], 1, 0, 3, 0.1)

array([0.50000004, 0.49999996])